In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat


planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)


In [6]:

from dotenv import load_dotenv

from langchain_community.utilities import GoogleSerperAPIWrapper

from autogen_ext.tools.http import HttpTool


os.environ['SERPER_API_KEY']='bead05022450578faa7498f4c90d85e534c372e0'


search_tool_wrapper = GoogleSerperAPIWrapper(type='search')

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found."  



In [3]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [4]:
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [7]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web],
    reflect_on_tool_use=True,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [8]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100



In [9]:

data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided (percentage_change_tool).

        If you have not seen the data, ask for it.

    ''',
)

# Termination Condition

In [10]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_message_termination

In [11]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [20]:
planning_agent.description

'An agent for planning tasks, this agent should be the first to engage when given a new task.'

In [13]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [15]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [14]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To provide the information requested, we need to perform the following tasks:

1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. Gather the total rebounds statistics for that player during the 2007-2008 and 2008-2009 NBA seasons.
3. Calculate the percentage change in total rebounds between these two seasons.

Let's proceed by delegating these tasks:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds for this player during the 2007-2008 NBA season.
3. WebSearchAgent: Find the total rebounds for this player during the 2008-2009 NBA season.
4. DataAnalystAgent: Calculate the percentage c

TaskResult(messages=[TextMessage(id='ea590aef-c725-42a7-bb2a-461aec7e78eb', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 5, 7, 38, 950020, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='47e5e5e6-77ff-4568-9a6f-2841f0976769', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=194), metadata={}, created_at=datetime.datetime(2025, 7, 13, 5, 7, 43, 382291, tzinfo=datetime.timezone.utc), content="To provide the information requested, we need to perform the following tasks:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. Gather the total rebounds statistics for that player during the 2007-2008 and 2008-2009 NBA seasons.\n3. Calculate the percentage change in t

In [16]:
# With real web search


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your query, we'll follow these steps:

1. Identify the Miami Heat player with the highest points in the 2006-2007 season. 
2. Find the total rebounds for this player in the 2007-2008 season.
3. Find the total rebounds for this player in the 2008-2009 season.
4. Calculate the percentage change in total rebounds between the two seasons.

Let's start with the first task:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 season.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_Tyg4NTSXboxkA9ZI4fJ1kM1A', arguments='{"query":"Miami Heat player highest points 2006-2007 season"}', name='search_web')]
---------- ToolCallExecutionEvent (WebSe

TaskResult(messages=[TextMessage(id='0a097eb6-6499-43ba-83ee-9d2efb57c16c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 5, 28, 23, 240555, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='9edc06aa-87b8-4b9e-87e5-5b711bc9e422', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=117), metadata={}, created_at=datetime.datetime(2025, 7, 13, 5, 28, 26, 301054, tzinfo=datetime.timezone.utc), content="To address your query, we'll follow these steps:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 season. \n2. Find the total rebounds for this player in the 2007-2008 season.\n3. Find the total rebounds for this player in the 2008-2009 season.\n4. Calculate the percentage change i